In [6]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import altair as alt
from IPython.display import Image
import requests
import json
from collections import Counter
import requests_oauthlib
from requests_oauthlib import OAuth2Session
import inputs
from dotenv import load_dotenv

#### Ejecutamos el archivo y vemos su head

In [ ]:
df = pd.read_csv('Countries_usefulFeatures.csv', encoding='latin-1')
df.rename(columns={'Country_Region': 'Country'}, inplace=True)
#Renombro la columna de pais por comodidad 
#df.drop(df.index[47])
df.head()

In [ ]:
boats=df.iloc[47],df.iloc[102]
df.drop(df.index[[47,102]], inplace=True)
#Descubro en las filas 47 y 102 dos barcos por lo que los descarto ya que no son paises, si encuentro algun barco mas lo agregare.

### ¿Cuantas filas y columnas hay?

In [ ]:
nRow, nCol = df.shape
print(f'Hay {nRow} filas y {nCol} columnas')

### ¿Cuanta data se ha perdido?

In [ ]:
df.isnull().sum()

### Solo 26 paises no registraron la primera muerte por Covid19 y se desconoce el tipo de confinamiento que han aplicado 30 paises.

In [ ]:
fullLock=df[df['Lockdown_Type']=='Full']
fullLock.shape[0]
print(f"El numero de paises que han aplicado una cuarentena total es de {fullLock.shape[0]} de {df.shape[0]} paises que se han cogido para el estudio")

In [ ]:
late=df[['Country','Tourism','Lockdown_Date','Lockdown_Type']].sort_values(by='Lockdown_Date', ascending=False)
late.dropna(inplace =True)
print(f"El primer pais en aplicar la cuarentena fue {late.iloc[-1][0]} y el ultimo {late.iloc[0][0]}.")

In [ ]:
mperj=df[['Country','Tourism','Lockdown_Date','Lockdown_Type']].sort_values(by='Tourism', ascending=False)
mperj



### En el rango de una semana el top 3 mundial de visitantes turisticos tuvieron que aplicar cuarentena, de los cuales Francia y España tuvieron que aplicarla del tipo total, siendo un desastre para sus economias


In [ ]:
menper=df[['Country','Longtitude','Lockdown_Date']]
menper.sort_values(by='Longtitude', ascending= True)

In [ ]:
from IPython.display import Image
Image(filename='../Covid19MAD/Outputs/captura_de_pantalla.png')

### Utilizo la logica de nuestra flamante vicepresidente para comprobar si se utilizo este argumento para aplicar cuarentenas, tras comprobar el resultado parece que no hay diferencias intrinsecas.

In [ ]:
df['Date_FirstConfirmedCase'] = pd.to_datetime(df['Date_FirstConfirmedCase'])
df['Date_FirstFatality']= pd.to_datetime(df['Date_FirstFatality'])
df['Lockdown_Date']=pd.to_datetime(df['Lockdown_Date'])
df['Lethal_lapse']= df['Date_FirstFatality']-df['Date_FirstConfirmedCase']

In [ ]:

df['Lethal_lapse']

#Asignamos la nueva columna con los dias transcurridos desde el primer caso confirmado hasta la primera muerte y reorganizamos
premura=df.sort_values(by='Lethal_lapse', ascending=False)
premura=premura.dropna()
print(f"El pais que menos tardo en detectar un fallecido relacionado tras el primer caso fue {premura.iloc[-1][0]} y el que mas fue {premura.iloc[0][0]} .")
premura

In [ ]:
df['Days_to_Lockdown']= df['Date_FirstConfirmedCase']-df['Lockdown_Date']
columnas_a_ordernar=['Country','Population_Size','Tourism','Mean_Age','Date_FirstConfirmedCase','Date_FirstFatality','Lethal_lapse','Lockdown_Date','Days_to_Lockdown','Lockdown_Type']
nuevas_columnas=columnas_a_ordernar+(df.columns.drop(columnas_a_ordernar).tolist())
df=df[nuevas_columnas]
enclaustro=df.sort_values(by='Days_to_Lockdown', ascending=True)
enclaustro=enclaustro.dropna()
e=df.loc[df['Days_to_Lockdown']=='0 days']
e=[i for i in e['Country']]
z = ', '.join([str(elem) for elem in e])
print(f"Los paises que menos tardaron en enclaustrarse fueron {z} con cero dias y el que mas fue {enclaustro.iloc[0][0]} con {enclaustro.iloc[0][7]}.")
df

In [ ]:
df['Days_to_Lockdown'].dropna(inplace =True)
df['Days_to_Lockdown']=df['Days_to_Lockdown'].astype(int)

paisesprc=df[['Country','Days_to_Lockdown']]
paisesprc=df[['Country','Days_to_Lockdown','Date_FirstConfirmedCase']].groupby('Days_to_Lockdown').agg('max').reset_index()

alt.Chart(paisesprc).mark_bar().encode(
    x="Country",
    y="Days_to_Lockdown",
    color=alt.condition(
        alt.datum.Lockdown_Date > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    ),
    tooltip=['Country','Days_to_Lockdown','Date_FirstConfirmedCase']
).properties(height=400,width=600,title="¿Cuantos dias pasaron para el enclaustro?")
#Mostrar paises frente dias que tardaron en encerrarse

In [ ]:
grafica_turismo_lapso=df[['Date_FirstFatality','Country']].groupby('Date_FirstFatality').agg('max').reset_index()
#Paises agrupados por turismo vs dias que tardaron en encerrarse 
bar = alt.Chart(grafica_turismo_lapso).mark_bar().encode(
    x='Country',
    y='Date_FirstFatality',
    tooltip=['Country','Date_FirstFatality']
)

rule = alt.Chart(grafica_turismo_lapso).mark_rule(color='red').encode(
    y='mean(Date_FirstFatality):Q',
    tooltip=['Date_FirstFatality):Q']
)

(bar + rule).properties(height=400,width=600,title="yas")


In [ ]:
df['Relative Tourism']=(df['Tourism']/100)*df['Population_Size']
paisesmasturisticos=df[['Relative Tourism','Country']].sort_values(by='Relative Tourism')
#pip install collections-extended
print(f"El pais mas turistico percentualmente es {paisesmasturisticos.max()[1]} y el menor {paisesmasturisticos.min()[1]}")

In [ ]:
df.sort_values(by='Relative Tourism').min()

In [ ]:
#Probar, hightlight celdas planets.head(10).style.highlight_max(color='yellow').highlight_min(color = 'lightblue')

In [ ]:
Turismo_Diasencierro=df[['Days_to_Lockdown','Tourism','Country']].sort_values(by='Days_to_Lockdown')

Turismo_Diasencierro=Turismo_Diasencierro.dropna()
#Recopila dias hasta el encierro, nº turistas y nombre del pais recopilado por numeros de dia para el encierro

#                                             ITS API TIME!

In [2]:
import tweepy
import json
import os
from dotenv import load_dotenv
load_dotenv()


CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ACCESS_SECRET = os.getenv("ACCESS_SECRET")

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

user=api.me()
print(user.name,",", user.screen_name)
#Consigo imprimir mi nombre tras un esfuerzo que me debe haber costado un par de rizos

卡西亚 , nadie_me_tiene


In [7]:
listaf=[follower.name for follower in tweepy.Cursor(api.followers).items()]
#un par de migrañas despue consigo filtrar mi lista de seguidores(no duden en hacerme el favor de ampliarla)
print (f"Tengo {len(listaf)} seguidores asique sabre si me seguis o no")

Rate limit reached. Sleeping for: 390


Tengo 83 seguidores asique sabre si me seguis o no


In [18]:
CONSUMER_KEY = os.getenv("CONSUMER_KEY")
CONSUMER_SECRET = os.getenv("CONSUMER_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
ACCESS_SECRET = os.getenv("ACCESS_SECRET")

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
search='IronHack'
nrTweets=500
for tweet in tweepy.Cursor(api.search, search).items(nrTweets):
    try:
        print('Tweet liked')
        tweet.favorite()
        time.sleep(100)
    except tweepy.TweepError as e:
            print (e.reason)
    except StopIteration:
            break
#Peloteo, le doy like a los ultimos tweets de ironhack sin animo de lucro

Tweet liked
[{'code': 139, 'message': 'You have already favorited this status.'}]
Tweet liked
[{'code': 139, 'message': 'You have already favorited this status.'}]
Tweet liked
[{'code': 139, 'message': 'You have already favorited this status.'}]
Tweet liked


NameError: name 'time' is not defined

In [26]:

for tweet in tweepy.Cursor(api.search,
                       q='COVID19',
                       lang="es",
                       geo='40.416775,-3.703790, 510',
                       result_type='mixed').items(10):
    print(json.dumps(tweet._json, indent=2))
        
data = [[tweet.id, tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweepy.Cursor(api.search,
                       q='COVID19',
                       lang="es",
                       geo='40.416775,-3.703790, 510',
                       result_type='mixed').items(10)]
df2 = pd.DataFrame(data=data, columns=['id', 'date', 'user', 'tweet_text'])

{
  "created_at": "Sun Sep 13 23:56:17 +0000 2020",
  "id": 1305294259843723265,
  "id_str": "1305294259843723265",
  "text": "Informe #COVID19 | 13 de septiembre:\n\n\ud83d\udccdProcesamos 2981 muestras, 493 positivos, todos comunitarios y aislados.\n\n\ud83d\udccdLa\u2026 https://t.co/EmyHVONCe0",
  "truncated": true,
  "entities": {
    "hashtags": [
      {
        "text": "COVID19",
        "indices": [
          8,
          16
        ]
      }
    ],
    "symbols": [],
    "user_mentions": [],
    "urls": [
      {
        "url": "https://t.co/EmyHVONCe0",
        "expanded_url": "https://twitter.com/i/web/status/1305294259843723265",
        "display_url": "twitter.com/i/web/status/1\u2026",
        "indices": [
          117,
          140
        ]
      }
    ]
  },
  "metadata": {
    "result_type": "popular",
    "iso_language_code": "es"
  },
  "source": "<a href=\"https://mobile.twitter.com\" rel=\"nofollow\">Twitter Web App</a>",
  "in_reply_to_status_id": null,
  "in_

In [29]:
df2.drop['id'].inplaceTrue)


TypeError: 'method' object is not subscriptable

In [ ]:

def searchTweets(query, lang, geo):
    '''
    Perform a query search request to Twitter with Tweepy. Parameters:
    - query: include a word or a complex query using operators
    - lang: language ISO code 639-1
    - geo: latitude, longitude and km or miles
    
    Returns a dataframe with the columns: query, date, id, user and text.
    '''
    print('Creating an empty dataframe with the first 10 tweets')
    try:
        count = 0
        #First request
        while count < 1:
            tweets = tweepy.Cursor(api.search,
                               q=query,
                               lang=lang,
                               geo=geo,
                               result_type='mixed').items(10)


            data = [[query,tweet.created_at, tweet.id, tweet.user.screen_name, tweet.text] for tweet in tweets]

            df_tw = pd.DataFrame(data=data, columns=['query','date', 'id', 'user', 'tweet_text'])
            print('Here you have your first 10 tweets')
            print(df_tw)

            count += 1

        #Add more tweets
        while count >= 1:
            tweets = tweepy.Cursor(api.search,
                                       q=query,
                                       lang=lang,
                                       geo=geo,
                                       result_type='mixed',
                                       max_id=df_tw.id.min()).items(1500)


            data = [[query, tweet.id, tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]
            df2 = pd.DataFrame(data=data, columns=['query', 'id', 'date', 'user', 'tweet_text'])
            df_tw = df_tw.append(df2, sort=True)

            count += 1
            print('Adding 1500 to the dataframe')
            print('Your dataframe has a total of', df_tw.shape[0], 'tweets.')
            print('Oldest tweet is from', df_tw.date.min())

            return df_tw.to_csv('df_search.csv')
    except AttributeError:
        pass
    except NameError:
        pass



In [ ]:
df2=searchTweets('#Covid19', 'en', '40.416775,-3.703790, 510')

In [ ]:

def Sort_Covid():
    sort_list=Split_Tweets()
    cov= []
    numberofTweets=5
    for sentence in sort_list:
        for word in sentence:
            if word.startswith("Coronavirus") or ("Covid"):
                cov.append(word)
    return cov
Sort_Covid()

In [ ]:
get_auth()